## Week 1: Capstone Project Task 0
### Introduction
This task has 2 goals
1. Obtain data from the course website
1. Familiarize yourself with background  

    2.1 What do the data look like?

    2.2 Where do the data come from?

    2.3 Can you think of any other data sources that might help you in this project?

    2.4 What are the common steps in natural language processing?

    2.5 What are some common issues in the analysis of text data?

    2.6 What is the relationship between NLP and the concepts you have learned in the Specialization?

### 0. Environmet Settings

In [1]:
library(data.tree)
library(DiagrammeR)
library(igraph)
library(influenceR)
library(plyr)
library(SnowballC)
library(stringi)
library(tm)


Attaching package: ‘igraph’


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union



Attaching package: ‘influenceR’


The following objects are masked from ‘package:igraph’:

    betweenness, constraint


Loading required package: NLP



Download the google profanity word list from this websit <a href="https://github.com/RobertJGabriel/Google-profanity-words">https://github.com/RobertJGabriel/Google-profanity-words</a> using `git clone`, and the file is `list.txt` respectively. Move it to the same directory

In [2]:
profanity_words <- read.table("~/Soft/Rtest/JHU_capstone_project_data/list.txt", header = FALSE)

### 1. Obtaining data

In [3]:
# set work directory accordingly
setwd("~/Soft/Rtest/datasciencecoursera/10_Capstone_Project/")

# download the dataset
url = "https://d396qusza40orc.cloudfront.net/dsscapstone/dataset/Coursera-SwiftKey.zip"
#download.file(url, 
#              destfile = "data/Coursera-SwiftKey.zip"
#             )

# unzip the file
# unzip("./data/Coursera-SwiftKey.zip", exdir = "data/")

Read the files into R

In [4]:
# Deutsch
blogs_DE <- readLines("~/Soft/Rtest/JHU_capstone_project_data/final/de_DE/de_DE.blogs.txt"
                     , encoding = "UTF-8"
                     , skipNul = TRUE)
news_DE <- readLines("~/Soft/Rtest/JHU_capstone_project_data/final/de_DE/de_DE.news.txt"
                     , encoding = "UTF-8"
                     , skipNul = TRUE)
twitter_DE <- readLines("~/Soft/Rtest/JHU_capstone_project_data/final/de_DE/de_DE.twitter.txt"
                     , encoding = "UTF-8"
                     , skipNul = TRUE)

# Finnish
blogs_FI <- readLines("~/Soft/Rtest/JHU_capstone_project_data/final/fi_FI/fi_FI.blogs.txt"
                     , encoding = "UTF-8"
                     , skipNul = TRUE)
news_FI <- readLines("~/Soft/Rtest/JHU_capstone_project_data/final/fi_FI/fi_FI.news.txt"
                     , encoding = "UTF-8"
                     , skipNul = TRUE)
twitter_FI <- readLines("~/Soft/Rtest/JHU_capstone_project_data/final/fi_FI/fi_FI.twitter.txt"
                     , encoding = "UTF-8"
                     , skipNul = TRUE)

# Russian
blogs_RU <- readLines("~/Soft/Rtest/JHU_capstone_project_data/final/ru_RU/ru_RU.blogs.txt"
                     , encoding = "UTF-8"
                     , skipNul = TRUE)
news_RU <- readLines("~/Soft/Rtest/JHU_capstone_project_data/final/ru_RU/ru_RU.news.txt"
                     , encoding = "UTF-8"
                     , skipNul = TRUE)
twitter_RU <- readLines("~/Soft/Rtest/JHU_capstone_project_data/final/ru_RU/ru_RU.twitter.txt"
                     , encoding = "UTF-8"
                     , skipNul = TRUE)

# English
blogs_US <- readLines("~/Soft/Rtest/JHU_capstone_project_data/final/en_US/en_US.blogs.txt"
                     , encoding = "UTF-8"
                     , skipNul = TRUE)
news_US <- readLines("~/Soft/Rtest/JHU_capstone_project_data/final/en_US/en_US.news.txt"
                     , encoding = "UTF-8"
                     , skipNul = TRUE)
twitter_US <- readLines("~/Soft/Rtest/JHU_capstone_project_data/final/en_US/en_US.twitter.txt"
                     , encoding = "UTF-8"
                     , skipNul = TRUE)


### 2.1 What do the data look like?
We try to show 3 characteristics of the data
- dendrogram tree of the data files
- number of lines by type and by country
- size of data sets by type and by country
- wordcounts by type and by country

In [5]:
dir("data/final")

character(0)

#### dendrogram tree of the data files

In [6]:
file_list <- c(
    "~/Soft/Rtest/JHU_capstone_project_data/final/de_DE/de_DE.blogs.txt",
    "~/Soft/Rtest/JHU_capstone_project_data/final/de_DE/de_DE.news.txt",
    "~/Soft/Rtest/JHU_capstone_project_data/final/de_DE/de_DE.twitter.txt",
    "~/Soft/Rtest/JHU_capstone_project_data/final/fi_FI/fi_FI.blogs.txt",
    "~/Soft/Rtest/JHU_capstone_project_data/final/fi_FI/fi_FI.news.txt",
    "~/Soft/Rtest/JHU_capstone_project_data/final/fi_FI/fi_FI.twitter.txt",
    "~/Soft/Rtest/JHU_capstone_project_data/final/ru_RU/ru_RU.blogs.txt",
    "~/Soft/Rtest/JHU_capstone_project_data/final/ru_RU/ru_RU.news.txt",
    "~/Soft/Rtest/JHU_capstone_project_data/final/ru_RU/ru_RU.twitter.txt",
    "~/Soft/Rtest/JHU_capstone_project_data/final/en_US/en_US.blogs.txt",
    "~/Soft/Rtest/JHU_capstone_project_data/final/en_US/en_US.news.txt",
    "~/Soft/Rtest/JHU_capstone_project_data/final/en_US/en_US.twitter.txt"
)

In [7]:
file_tree <- lapply(strsplit(file_list, "/"), function(z) as.data.frame(t(z)))
file_tree <- rbind.fill(file_tree)
file_tree$pathString <- apply(file_tree, 1, function(z) paste(trimws(na.omit(z)), collapse = "/"))
my_tree <- data.tree::as.Node(file_tree)
my_tree # unix tree-command like style
#plot(my_tree) # more elegant style

                                   levelName
1  ~                                        
2   °--Soft                                 
3       °--Rtest                            
4           °--JHU_capstone_project_data    
5               °--final                    
6                   ¦--de_DE                
7                   ¦   ¦--de_DE.blogs.txt  
8                   ¦   ¦--de_DE.news.txt   
9                   ¦   °--de_DE.twitter.txt
10                  ¦--fi_FI                
11                  ¦   ¦--fi_FI.blogs.txt  
12                  ¦   ¦--fi_FI.news.txt   
13                  ¦   °--fi_FI.twitter.txt
14                  ¦--ru_RU                
15                  ¦   ¦--ru_RU.blogs.txt  
16                  ¦   ¦--ru_RU.news.txt   
17                  ¦   °--ru_RU.twitter.txt
18                  °--en_US                
19                      ¦--en_US.blogs.txt  
20                      ¦--en_US.news.txt   
21                      °--en_US.twitter.txt

In [16]:
saveRDS(my_tree, file = "data/my_tree.RDS")

#### Number of lines by type and by country

In [8]:
length_df <- data.frame(
    country = "DE"
    , blogs = c(length(blogs_DE))
    , news = c(length(news_DE))
    , twitter = c(length(twitter_DE))
)
length_df <- rbind(length_df, c("FI", length(blogs_FI), length(news_FI), length(twitter_FI)))
length_df <- rbind(length_df, c("RU", length(blogs_RU), length(news_RU), length(twitter_RU)))
length_df <- rbind(length_df, c("US", length(blogs_US), length(news_US), length(twitter_US)))

So that the data sets' length (lines) of each type (blogs, news, twitter) by country is

In [9]:
print(length_df)

  country  blogs    news twitter
1      DE 371440  244743  947774
2      FI 439785  485758  285214
3      RU 337100  196360  881414
4      US 899288 1010242 2360148


In [17]:
saveRDS(length_df, file = "data/length_df.RDS")

#### Size of data sets by type and by country

In [10]:
# Deutsch
blogs_size_DE <- file.info("~/Soft/Rtest/JHU_capstone_project_data/final/de_DE/de_DE.blogs.txt")$size/(1024.0^2)
news_size_DE <- file.info("~/Soft/Rtest/JHU_capstone_project_data/final/de_DE/de_DE.news.txt")$size/(1024.0^2)
twitter_size_DE <- file.info("~/Soft/Rtest/JHU_capstone_project_data/final/de_DE/de_DE.twitter.txt")$size/(1024.0^2)

# Finnish
blogs_size_FI <- file.info("~/Soft/Rtest/JHU_capstone_project_data/final/fi_FI/fi_FI.blogs.txt")$size/(1024.0^2)
news_size_FI <- file.info("~/Soft/Rtest/JHU_capstone_project_data/final/fi_FI/fi_FI.news.txt")$size/(1024.0^2)
twitter_size_FI <- file.info("~/Soft/Rtest/JHU_capstone_project_data/final/fi_FI/fi_FI.twitter.txt")$size/(1024.0^2)

# Russian
blogs_size_RU <- file.info("~/Soft/Rtest/JHU_capstone_project_data/final/ru_RU/ru_RU.blogs.txt")$size/(1024.0^2)
news_size_RU <- file.info("~/Soft/Rtest/JHU_capstone_project_data/final/ru_RU/ru_RU.news.txt")$size/(1024.0^2)
twitter_size_RU <- file.info("~/Soft/Rtest/JHU_capstone_project_data/final/ru_RU/ru_RU.twitter.txt")$size/(1024.0^2)

# English
blogs_size_US <- file.info("~/Soft/Rtest/JHU_capstone_project_data/final/en_US/en_US.blogs.txt")$size/(1024.0^2)
news_size_US <- file.info("~/Soft/Rtest/JHU_capstone_project_data/final/en_US/en_US.news.txt")$size/(1024.0^2)
twitter_size_US <- file.info("~/Soft/Rtest/JHU_capstone_project_data/final/en_US/en_US.twitter.txt")$size/(1024.0^2)

In [11]:
size_df <- data.frame(
    country = "DE"
    , blogs = round(blogs_size_DE, 2)
    , news = round(news_size_DE, 2)
    , twitter = round(twitter_size_DE,2)
)
size_df <- rbind(size_df, c("FI", round(blogs_size_FI, 2), round(news_size_FI,2), round(twitter_size_FI,2)))
size_df <- rbind(size_df, c("RU", round(blogs_size_RU, 2), round(news_size_RU,2), round(twitter_size_RU,2)))
size_df <- rbind(size_df, c("US", round(blogs_size_US, 2), round(news_size_US,2), round(twitter_size_US,2)))

So that the data sets' size (MBs) of each type (blogs, news, twitter) by country is

In [12]:
print(size_df)

  country  blogs   news twitter
1      DE   81.5  91.16   72.08
2      FI 103.48  89.87   24.16
3      RU 111.44 113.48  100.31
4      US 200.42 196.28  159.36


In [18]:
saveRDS(size_df, file = "data/size_df.RDS")

#### Wordcounts by type and by country

In [13]:
blogs_words_DE <-  sum(sapply(gregexpr("\\S+", blogs_DE), length))
news_words_DE <-  sum(sapply(gregexpr("\\S+", news_DE), length))
twitter_words_DE <-  sum(sapply(gregexpr("\\S+", twitter_DE), length))

blogs_words_FI <-  sum(sapply(gregexpr("\\S+", blogs_FI), length))
news_words_FI <-  sum(sapply(gregexpr("\\S+", news_FI), length))
twitter_words_FI <-  sum(sapply(gregexpr("\\S+", twitter_FI), length))

blogs_words_RU <-  sum(sapply(gregexpr("\\S+", blogs_RU), length))
news_words_RU <-  sum(sapply(gregexpr("\\S+", news_RU), length))
twitter_words_RU <-  sum(sapply(gregexpr("\\S+", twitter_RU), length))

blogs_words_US <-  sum(sapply(gregexpr("\\S+", blogs_US), length))
news_words_US <-  sum(sapply(gregexpr("\\S+", news_US), length))
twitter_words_US <-  sum(sapply(gregexpr("\\S+", twitter_US), length))

In [14]:
words_df <- data.frame(
    country = "DE"
    , blogs = round(blogs_words_DE, 2)
    , news = round(news_words_DE, 2)
    , twitter = round(twitter_words_DE,2)
)
words_df <- rbind(words_df, c("FI", blogs_words_FI, news_words_FI, twitter_words_FI))
words_df <- rbind(words_df, c("RU", blogs_words_RU, news_words_RU, twitter_words_RU))
words_df <- rbind(words_df, c("US", blogs_words_US, news_words_US, twitter_words_US))

So that the data sets' word counts of each type (blogs, news, twitter) by country is

In [15]:
print(words_df)

  country    blogs     news  twitter
1      DE 12653019 13219287 11803491
2      FI 12731005 10445964  3152758
3      RU  9405378  9115829  9223841
4      US 37334131 34372530 30373583


In [19]:
saveRDS(words_df, file = "data/words_df.RDS")

### 2.2 Where do the data come from?

The data come from **SwiftKey**, and the url is 
<a href="https://d396qusza40orc.cloudfront.net/dsscapstone/dataset/Coursera-SwiftKey.zip">https://d396qusza40orc.cloudfront.net/dsscapstone/dataset/Coursera-SwiftKey.zip</a>

### 2.3 Can you think of any other data sources that might help you in this project?

Yes. For example, the WIKIPEDIA is a good text corpus for NLP

### 2.4 What are the common steps in natural language processing?
According to <a href="https://medium.com/@ageitgey/natural-language-processing-is-fun-9a0bff37854e">https://medium.com/@ageitgey/natural-language-processing-is-fun-9a0bff37854e</a>, generally speaking NLP takes the common steps
- Sentence Segmentation
- Word Tokenization
- Predicting Parts of Speech for Each Token
- Text Lemmatization
- Stop Words Identification
- Dependency Parsing
- Finding Noun Phrases
- Named Entity Recognition (NER)
- Conference Resolution

### 2.5 What are some common issues in the analysis of text data?
The common issues in the analysis of text data are:
- Grammar mistakes: e.g. Father and Fahter
- Capital and Lower words: e.g. World and world
- Words of similar meanings: e.g. UK, Britan, GB ...

### 2.6 What is the relationship between NLP and the concepts you have learned in the Specialization?

NLP use algorithms similar to those used in ML, for example, such as statistical inference

The difference is mainly that they deal with varing objects, e.g. housing price prediction model mainly focuses on price which is number, and NLP mainly focuses on letter / words / sentence which is character.

## Capstone Project Task 1

**Tasks to accomplish**

1. Tokenization - identifying appropriate tokens such as words, punctuation, and numbers. Writing a function that takes a file as input and returns a tokenized version of it.
1. Profanity filtering - removing profanity and other words you do not want to predict.

**Tips, tricks, and hints**

1. Loading the data in. This dataset is fairly large. We emphasize that you don't necessarily need to load the entire dataset in to build your algorithms (see point 2 below). At least initially, you might want to use a smaller subset of the data. Reading in chunks or lines using R's readLines or scan functions can be useful. You can also loop over each line of text by embedding readLines within a for/while loop, but this may be slower than reading in large chunks at a time. Reading pieces of the file at a time will require the use of a file connection in R. For example, the following code could be used to read the first few lines of the English Twitter dataset:con <- file("en_US.twitter.txt", "r") readLines(con, 1) ## Read the first line of text readLines(con, 1) ## Read the next line of text readLines(con, 5) ## Read in the next 5 lines of text close(con) ## It's important to close the connection when you are done. See the connections help page for more information.
2. Sampling. To reiterate, to build models you don't need to load in and use all of the data. Often relatively few randomly selected rows or chunks need to be included to get an accurate approximation to results that would be obtained using all the data. Remember your inference class and how a representative sample can be used to infer facts about a population. You might want to create a separate sub-sample dataset by reading in a random subset of the original data and writing it out to a separate file. That way, you can store the sample and not have to recreate it every time. You can use the rbinom function to "flip a biased coin" to determine whether you sample a line of text or not.

### 1. Generate a random sample

In [ ]:
sample_blogs_US <- blogs_US[sample(1:length(blogs_US), 10000)]
sample_news_US <- news_US[sample(1:length(news_US), 10000)]
sample_twitter_US <- twitter_US[sample(1:length(twitter_US), 10000)]

sample_sum_US <- c(sample_blogs_US, sample_news_US, sample_twitter_US)

sum_US <- c(blogs_US, news_US, twitter_US)

### 2. Build the corpus

In [ ]:
# Specify the source to be character vectors
sample_sum_US <- Corpus(VectorSource(sample_sum_US))
sum_US <- Corpus(VectorSource(sum_US))

In [ ]:
# convert to lower case
sample_sum_US <- tm_map(
    sample_sum_US
    , content_transformer(tolower)
#    , lazy = TRUE
)
sum_US <- tm_map(
                sum_US
                , content_transformer(tolower)
                #    , lazy = TRUE
                )

In [ ]:
# sample_sum_US

# remove punction
sample_sum_US <- tm_map(sample_sum_US, content_transformer(removePunctuation))

# remove Numbers
sample_sum_US <- tm_map(sample_sum_US, content_transformer(removeNumbers))

# remove URLs
removeURL <- function(x) gsub("http[[:alnum:]]*", "", x)
sample_sum_US <- tm_map(sample_sum_US, content_transformer(removeURL))

# remove whitespace
sample_sum_US <- tm_map(sample_sum_US, content_transformer(stripWhitespace))

# remove stopwords with english
sample_sum_US <- tm_map(sample_sum_US, removeWords, stopwords("english"))

# remove profanity words
sample_sum_US <- tm_map(sample_sum_US, removeWords, profanity_words[,1])

#stemDocument
sample_sum_US <- tm_map(sample_sum_US, stemDocument)

In [ ]:
# sum_US

# remove punction
sum_US <- tm_map(sum_US, content_transformer(removePunctuation))

# remove Numbers
sum_US <- tm_map(sum_US, content_transformer(removeNumbers))

# remove URLs
# removeURL <- function(x) gsub("http[[:alnum:]]*", "", x)
sum_US <- tm_map(sum_US, content_transformer(removeURL))

# remove whitespace
sum_US <- tm_map(sum_US, content_transformer(stripWhitespace))

# remove stopwords with english
sum_US <- tm_map(sum_US, removeWords, stopwords("english"))

# remove profanity words
sum_US <- tm_map(sum_US, removeWords, profanity_words[,1])

#stemDocument
sum_US <- tm_map(sum_US, stemDocument)

In [ ]:
# Save the data
writeLines(sample_sum_US, "~/Soft/Rtest/JHU_capstone_project_data/final/sample_sum_US.txt")
writeLines(sum_US, "~/Soft/Rtest/JHU_capstone_project_data/final/sum_US.txt")

#### Question 3

What is the length of the longest line seen in any of the three en_US data sets?

In [ ]:
con <- file("~/Soft/Rtest/JHU_capstone_project_data/final/en_US/en_US.blogs.txt")
lenn <- nchar(readLines(con = con))
max_len_blogs_US <- max(lenn)
close(con)
max_len_blogs_US

In [ ]:
con <- file("~/Soft/Rtest/JHU_capstone_project_data/final/en_US/en_US.twitter.txt", "r")
lenn <- nchar(readLines(con = con))
max_len_twitter_US <- max(lenn)
close(con)
max_len_twitter_US

In [ ]:
con <- file("~/Soft/Rtest/JHU_capstone_project_data/final/en_US/en_US.news.txt", "r")
lenn <- nchar(readLines(con = con))
max_len_news_US <- max(lenn)
close(con)
max_len_news_US

#### Question 4

In the en_US twitter data set, if you divide the number of lines where the word “love” (all lowercase) occurs by the number of lines the word “hate” (all lowercase) occurs, about what do you get?

In [ ]:
con <- file("~/Soft/Rtest/JHU_capstone_project_data/final/en_US/en_US.twitter.txt", "r")
lovec <- grepl(".love."#".[Ll][Oo][Vv][Ee]."
              , readLines(con)
               , ignore.case = FALSE
              )
lovec <- sum(lovec)
close(con)

In [ ]:
con <- file("~/Soft/Rtest/JHU_capstone_project_data/final/en_US/en_US.twitter.txt", "r")
hatec <- grepl(".hate."#".[Hh][Aa][Tt][Ee]."
               , readLines(con)
               , ignore.case = FALSE
              )
hatec <- sum(hatec)
hatec
close(con)

In [ ]:
lovec/hatec

#### Question 5: 
The one tweet in the en_US twitter data set that matches the word “biostats” says what?

In [ ]:
con <- file("~/Soft/Rtest/JHU_capstone_project_data/final/en_US/en_US.twitter.txt", "r")
readfile <- readLines(con)
readfile[grepl(".biostats.", readfile, ignore.case = FALSE) == TRUE]
close(con)

#### Question 6:
How many tweets have the exact characters "A computer once beat me at chess, but it was no match for me at kickboxing". (I.e. the line matches those characters exactly.)

In [ ]:
con <- file("~/Soft/Rtest/JHU_capstone_project_data/final/en_US/en_US.twitter.txt", "r")
readfile <- readLines(con)
readfile[
    grepl("A computer once beat me at chess, but it was no match for me at kickboxing"
         , readfile
         , ignore.case = FALSE) == TRUE
]
close(con)